In [9]:
# load the csv with the data
import pandas as pd
import re
import os

RESULTS_DIR = './../datasets_reduced/revision/results/'

INPUT_FILE = 'dataset_random_baseline.jsonl'
OUTPUT_FILE = RESULTS_DIR + 'dataset_random_baseline_with_completions.csv'

# cd with jupyter into the directory with the results
os.chdir(RESULTS_DIR)



FileNotFoundError: [Errno 2] No such file or directory: './../datasets_reduced/revision/results/'

In [12]:
# data
data = pd.read_json('dataset_random_baseline.jsonl', lines=True)

# Some config
MULTI_EDGE = False
model = "gpt-4"


In [13]:
# select data, i.e. fixed_prompt column. select only rows where completion_type = ['Add_node']
count = 0
for index, row in data.iterrows():
    if row['completion_type'] == "['Add_node']" and re.match(r".*'changeType': 'Add', 'type': 'object'.*",row['completion']):
        count += 1

print(f"There are {count} rows with completion_type = ['Add_node']")

There are 51 rows with completion_type = ['Add_node']


Run the completion with gpt-4

In [15]:
# cd back to the root (because the GPT_runner.py script expects to be in the root for access to secret api keys etc.)
os.chdir("./../../../lm2eo-main")

# define the prompts

from GPT_runner import sample_edge_gpt_chat, CHAT_MODEL_INSTRUCTION

# select data, i.e. fixed_prompt column. select only rows where completion_type = ['Add_node']
count = 0
for index, row in data.iterrows():
    if row['completion_type'] == "['Add_node']" and re.match(r".*'changeType': 'Add', 'type': 'object'.*",row['completion']):
        total_tokens, completion_tokens, completion_string = sample_edge_gpt_chat(model, row['prompt'], CHAT_MODEL_INSTRUCTION, MULTI_EDGE)
        count += 1
        # set in data file
        data.at[index, 'completion_generated'] = completion_string
        data.at[index, 'completion_tokens'] = completion_tokens
        data.at[index, 'total_tokens'] = total_tokens

# save the data
data.to_csv(OUTPUT_FILE, index=False)



Using azure.
